In [1]:
import pandas as pd
from pandas_datareader import data, wb
import matplotlib as mpl
from mplfinance.original_flavor import candlestick_ohlc
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot, iplot
import matplotlib.dates as dates
import datetime


In [2]:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2022, 1, 1)
unused = "unused"  #i just want to check here if this variable will be marked as unused

In [3]:
df=data.DataReader('F', 'yahoo', start, end)

df['Dates'] = dates.date2num(df.index)

period9_high = df['High'].rolling(window=9).max()
period9_low = df['Low'].rolling(window=9).min()
df['Tenkan-sen'] = (period9_high + period9_low) / 2

period26_high = df['High'].rolling(window=26).max()
period26_low =  df['Low'].rolling(window=26).min()
df['Kijun-sen'] = (period26_high + period26_low) / 2

df['Senkou-span_1'] = ((df['Tenkan-sen'] + df['Kijun-sen']) / 2).shift(26)

period52_high = df['High'].rolling(window=52).max()
period52_low = df['Low'].rolling(window=52).min()
df['Senkou-span_2'] = ((period52_high + period52_low) / 2).shift(26)

df['Chikou-span'] = df['Close'].shift(-26)

df

,High,Low,Open,Close,Volume,Adj Close,Dates,Tenkan-sen,Kijun-sen,Senkou-span_1,Senkou-span_2,Chikou-span
Date,,,,,,,,,,,,
2019-01-02,8.020000,7.480000,7.530000,7.900000,47494400.0,7.092320,17898.0,NaN,NaN,NaN,NaN,8.39
2019-01-03,7.990000,7.780000,7.970000,7.780000,39172400.0,6.984588,17899.0,NaN,NaN,NaN,NaN,8.33
2019-01-04,8.120000,7.850000,7.910000,8.080000,43039800.0,7.253916,17900.0,NaN,NaN,NaN,NaN,8.46
2019-01-07,8.370000,8.030000,8.110000,8.290000,40729400.0,7.442447,17903.0,NaN,NaN,NaN,NaN,8.41
2019-01-08,8.540000,8.350000,8.420000,8.370000,45644000.0,7.514268,17904.0,NaN,NaN,NaN,NaN,8.42
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,20.889999,20.100000,20.260000,20.799999,59651800.0,20.353836,18988.0,20.075000,20.05,18.8200,16.565,NaN
2021-12-28,21.059999,20.590000,20.660000,20.760000,53020500.0,20.314695,18989.0,20.075000,20.05,18.8625,16.565,NaN
2021-12-29,20.770000,20.450001,20.709999,20.559999,37883000.0,20.118984,18990.0,20.075000,20.05,18.8625,16.565,NaN


In [4]:
#candlestick chart data for our first series to plot
#also is the candlestick element itself
data = [{
    'type' : 'candlestick',
    'open' : df.Open,
    'high' : df.High,
    'low' : df.Low,
    'close' : df.Close,
    'x' : df.index,
    'yaxis' : 'y2',
    'name' : 'Ford chart',
}
]
#empty dictionary for later use to hold settings and layout options
layout = dict()

#main chart fig consists of data and layout settings
fig = {'data' : data, 'layout' : layout}

In [5]:
# Assign various seeting and choices - background colour, range selector etc
fig['layout']['plot_bgcolor'] = 'rgb(44, 48, 56)'
fig['layout']['xaxis'] = {'rangeselector' : {'visible' : True}}
fig['layout']['yaxis'] = {
                'domain' : [0, 0.2], 
                'showticklabels' : False 
                        }
fig['layout']['yaxis2'] = {'domain' : [0.2, 0.8]}
fig['layout']['legend'] = {
                'orientation' : 'h',
                'y' : 0.9,
                'x' : 0.3, 
                'yanchor' : 'bottom' 
                            }
fig['layout']['margin'] = {
                't' : 40,
                'b' : 40,
                'r' : 40,
                'l' : 40
                            }                            

In [6]:
#populate the rangeselector object with necessary settings
rangeselector = {
    'visible' : True,
    'x' : 0, 'y' : 0.9,
    'bgcolor' : 'rgba(148, 152, 161, 0.4)',
    'font' : {'size' : 13},
    'buttons' : [
        {'count' : 1, 'label' : 'reset', 'step' : 'all'},
        {'count' : 1, 'label' : '1yr', 'step' : 'year', 'stepmode' : 'backward'},
        {'count' : 3, 'label' : '3 mo', 'step' : 'month', 'stepmode' : 'backward'},
        {'count' : 1, 'label' : '1 mo', 'step' : 'month', 'stepmode' : 'backward'},
        {'step' : 'all'},
    ]
}

fig['layout']['xaxis']['rangeselector'] = rangeselector

In [7]:
#append the ichimoku elements to the plot

fig['data'].append({
                'x' : df['Tenkan-sen'].index, 'y' : df['Tenkan-sen'],
                'type' : 'scatter',
                'mode' : 'lines',
                'line' : {'width' : 1},
                'marker' : {'color' : '#de7b12'},
                'yaxis' : 'y2', 'name' : 'Tenkan-sen'
                    })
fig['data'].append( {
                'x' : df['Kijun-sen'].index, 'y' : df['Kijun-sen'],
                'type' : 'scatter', 'mode' : 'lines',
                'line' : {'width' : 1},
                'marker' : {'color' : '#1245de'},
                'yaxis' : 'y2', 'name' : 'Kijun-sen'
    })
fig['data'].append({
                'x' : df['Senkou-span_1'].index, 'y' : df['Senkou-span_1'],
                'type' : 'scatter', 'mode' : 'lines',
                'line' : {'width' : 1},
                'marker' : {'color' : '#228B22'},
                'yaxis' : 'y2', 'name' : 'Senkou-span 1'
})                   
fig['data'].append({
                'x' : df['Senkou-span_2'].index, 'y' : df['Senkou-span_2'],
                'type' : 'scatter', 'mode' : 'lines',
                'line' : {'width' : 1}, 'fill' : "tonexty",
                'marker' : {'color' : '#FF3341'},
                'yaxis' : 'y2', 'name' : 'Senkou-span 2'
})        
fig['data'].append({
                'x' : df['Chikou-span'].index, 'y' : df['Chikou-span'],
                'type' : 'scatter', 'mode' : 'lines',
                'line' : {'width' : 1},
                'marker' : {'color' : '#046e08'},
                'yaxis' : 'y2', 'name' : 'Chikou-span-span 1'
})        



In [8]:
#set colour list for candlesticks
"""colors = []
for i in range(len(df.Close)):
    if i != 0:
        if df.Close[1] > df.Close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)
""" #i wonder what's a purpose of this


'colors = []\nfor i in range(len(df.Close)):\n    if i != 0:\n        if df.Close[1] > df.Close[i-1]:\n            colors.append(INCREASING_COLOR)\n        else:\n            colors.append(DECREASING_COLOR)\n    else:\n        colors.append(DECREASING_COLOR)\n'

In [9]:
iplot(fig, filename = 'candlestick-ichimoku' )